# 2. Operaciones matemáticas con numpy

Vamos a realizar diferentes operaciones comunes de algebra lineal en numpy. Considera las siguientes operaciones:


Realicemos ahora la misma operación en numpy:
Puedes usar diferentes operadores para realizar operaciones con matrices
* `np.dot(a,b)` [docs](https://numpy.org/doc/stable/reference/generated/numpy.dot.html): Se utiliza para obtener el producto punto de dos vectores. Sin embargo cuando a o b son matrices, realiza una multiplicacion de matrices.
* `A @ B` [docs](https://numpy.org/doc/stable/reference/generated/numpy.dot.html): Si `A` y `B` son arreglos de numpy (`np.Array([algo])`), realiza una operacion de matrices regular. Recuerda que deben tener la dimensionalidad correcta para la operación.
* `np.multiply(a,b)`: Resulta en el producto hadamart.
* `A * B`: También corresponde a un producto hadamart, es decir una multiplicacion punto a punto. Lo mismo ocurre con los operadores `+`, `-`, y `/`. 

Recuerda que numpy permite realizar operaciones aun cuando las dimensiones no son identicas debido a que realiza un "broadcasting".

### Instrucciones:
Completa el siguiente codigo y compara el resultado de numpy con el analítico.

In [10]:
W = np.array([[2,3],
              [4,5]])
print(W.shape)
# TODO: inicializa x
x = np.array([5,
               6])
print(x.shape)
# TODO: Realiza la multiplicación de Wx
A = np.dot(W, x)
print(A.shape)
print("Resultado:")
print(A)

(2, 2)
(2,)
(2,)
Resultado:
[28 50]
